**S16** ¿Cuál es el usuario más quejumbroso por estado? Consideramos quejumbroso a un usuario que tiene más de 5 reviews y son todas de 2 estrellas o menos.

In [1]:
!pip install pyspark==3.2.1

In [2]:
from pyspark.sql import *

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
sqlContext = SQLContext(sc)


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
# Change the file directories to yours, the files are in a public folder in google drive:
# https://drive.google.com/drive/folders/1CsvJs0xZ9SCLtG4ci2dNGCkhqnI-1DLS?usp=drive_link
df_user = sqlContext.read.csv('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/user.csv', header=True, inferSchema=True)
rdd_user = df_user.rdd

In [6]:
df_review = sqlContext.read.csv('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/review.csv', header=True, inferSchema=True)
rdd_review = df_review.rdd

In [7]:
df_business = sqlContext.read.csv('/content/drive/MyDrive/2022/Orga_de_Datos/Datasets/business.csv', header=True, inferSchema=True)
rdd_business = df_business.rdd

In [8]:
def check_numeric(x):
    try:
        float(x)
        return True
    except:
        return False

In [9]:
rdd_users_reviews = rdd_review.filter(lambda x: check_numeric(x.stars))\
  .map(lambda x: (x.user_id, x.business_id, float(x.stars)))\
  .map(lambda x: (x[0], ([x[1]], [x[2]])))\
  .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
  .filter(lambda x: (len(x[1][0]) > 5 and all(val <= 2 for val in x[1][1])))\
  .flatMap(lambda x: [(business_id, (x[0], x[1][1][index])) for index, business_id in enumerate(x[1][0])])\
  .groupByKey()\
  .mapValues(list)\
# Limpo el dataset (stars numéricos)
# Me quedo con los usuarios, business_id y stars del review
# Obtengo un rdd con (user_id, lista de business_id, lista de stars)
# Filtro los que tengan menos de 6 reviews o alguna con más de 2 estrellas
# Obtengo un rdd con (business_id, lista de tuplas (user_id, stars))

In [10]:
rdd_business_short = rdd_business.map(lambda x: (x.business_id, x.state))\
  .rightOuterJoin(rdd_users_reviews)\
  .map(lambda x: (x[1][0], x[1][1]))\
  .flatMap(lambda x: ([(x[0], user) for user in x[1]]))\
  .map(lambda x: ((x[0], x[1][0]) , 1))\
  .reduceByKey(lambda x,y: x+y)\
  .map(lambda x: (x[0][0], (x[0][1], x[1])))\
  .reduceByKey(lambda x,y: x if x[1] > y[1] else y)\
# Me quedo con los business_id y los estados de cada negocio
# Hago un join para agregar los usuarios con sus reviews al rdd
# Agrupo el rdd por estado, y en cada estado tengo los usuarios quejumbrosos
# Agrupo por tupla (estado, usuario) y sumo las ocurrencias de cada una de estas tuplas
# Paso al estado como key, y reduzco quedándome con el usuario más quejumbroso

In [11]:
for user in rdd_business_short.collect():
  print(f"el usuario más quejumbroso del estado {user[0]} tiene user_id: {user[1][0]}")

el usuario más quejumbroso del estado LA tiene user_id: xAV8XItZyjqQWYdQK3GJww
el usuario más quejumbroso del estado PA tiene user_id: ttyzgc93vTrwIYoDY0UmXw
el usuario más quejumbroso del estado TN tiene user_id: HYKIOKPr2KQbk-72V4ANVg
el usuario más quejumbroso del estado ID tiene user_id: KBSglJcN61YIWleS04TJtQ
el usuario más quejumbroso del estado DE tiene user_id: T3oldmX-JGVMql9GjEAEMg
el usuario más quejumbroso del estado AZ tiene user_id: qpmzZ5G7x-IMSTb2VYeJ_Q
el usuario más quejumbroso del estado MO tiene user_id: _HGW2i7EdvkKH04JtByGFQ
el usuario más quejumbroso del estado IL tiene user_id: Q0GISGD6Vy-u3lTFuHU_5Q
el usuario más quejumbroso del estado IN tiene user_id: CLg0CzbxOK1qfvzG6rP_Zg
el usuario más quejumbroso del estado CA tiene user_id: gSbWYsI9D_CLUsheOo_ttw
el usuario más quejumbroso del estado AB tiene user_id: A2KEoDZ7qqO_SDImvoFJrw
el usuario más quejumbroso del estado FL tiene user_id: 2uyyLfTPfF9vckrg5SHS6g
el usuario más quejumbroso del estado NJ tiene user_